In [2]:
import os
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, ConfusionMatrixDisplay, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [4]:
# Carregar os dados
X_treino = np.load(os.path.join('X_treino_final.npy'), allow_pickle=True)
y_treino = np.load(os.path.join('y_treino_final.npy'))
X_teste = np.load(os.path.join('X_teste_final.npy'), allow_pickle=True)
y_teste = np.load(os.path.join('y_teste_final.npy'))

In [5]:
# Verificar o tamanho dos arquivos
print(f"Shape de X_treino: {X_treino.shape}")
print(f"Shape de y_treino: {y_treino.shape}")
print(f"Shape de X_teste: {X_teste.shape}")
print(f"Shape de y_teste: {y_teste.shape}")

Shape de X_treino: (69600, 59)
Shape de y_treino: (69600,)
Shape de X_teste: (17400, 59)
Shape de y_teste: (17400,)


In [7]:
# --- 2. Definir o Pipeline para o Modelo SVM ---
# O pipeline inclui o StandardScaler para normalização dos dados e o classificador SVC.
# O scaler é crucial para o bom desempenho do SVM.
# Usamos with_mean=False no StandardScaler se os dados de entrada forem esparsos (CSR matrix),
# pois matrizes esparsas não suportam centralização (subtração da média) de forma eficiente.
# Se seus dados são densos, with_mean=True (padrão) é geralmente o ideal.
scaler = StandardScaler(with_mean=False if isinstance(X_treino, (np.ndarray, np.matrix)) and X_treino.ndim > 1 and np.sum(X_treino) == 0 else True)

svm = SVC(probability=True, random_state=42) # probability=True permite o predict_proba, se necessário. random_state para reprodutibilidade.

pipe = Pipeline([
    ("scaler", scaler),
    ("svm", svm)
])

In [ ]:
# --- 3. Definir os Parâmetros para GridSearchCV (Otimização de Hiperparâmetros) ---
# Vamos começar com o kernel 'rbf' (Radial Basis Function), que é um bom ponto de partida
# para muitos problemas de classificação não linear.
params_rbf = {
    "svm__C": [0.1, 1, 10, 100, 1000],          # Parâmetro de regularização
    "svm__gamma": [1e-4, 1e-3, 1e-2, 0.1, 1, 'scale', 'auto'], # Coeficiente do kernel
    "svm__kernel": ['rbf']
}

# Você pode testar outros kernels também, como 'linear' ou 'sigmoid'
# params_linear = {
#     "svm__C": [0.1, 1, 10, 100],
#     "svm__kernel": ['linear']
# }

# params_sigmoid = {
#     "svm__C": [0.1, 1, 10, 100, 1000],
#     "svm__gamma": [2e-5, 2e-3, 2e-1, "auto", "scale"],
#     "svm__coef0": [0, 1, 10], # Apenas para kernel 'poly' e 'sigmoid'
#     "svm__kernel": ['sigmoid']
# }

# Escolha o conjunto de parâmetros que deseja otimizar.
# Para este exemplo, vou focar no kernel 'rbf' inicialmente.
param_grid = params_rbf

modelo_svm = GridSearchCV(
    pipe,
    param_grid,
    cv=5,                  # 5-fold cross-validation
    scoring='accuracy',    # Métrica principal para otimização
    n_jobs=-1,             # Usa todos os núcleos disponíveis da CPU
    verbose=2              # Exibe o progresso do GridSearchCV
)

print("\n--- Iniciando Treinamento e Otimização do Modelo SVM ---")
print(f"Parâmetros a serem testados: {param_grid}")
modelo_svm.fit(X_treino, y_treino)

# --- 4. Exibir os Melhores Parâmetros e Pontuação de Validação Cruzada ---
print("\n--- Resultados da Otimização GridSearchCV ---")
print(f"Melhores hiperparâmetros encontrados: {modelo_svm.best_params_}")
print(f"Melhor acurácia da validação cruzada: {modelo_svm.best_score_:.4f}")

# Obter o melhor modelo treinado (com os melhores hiperparâmetros)
best_svm_model = modelo_svm.best_estimator_

In [ ]:
# --- 5. Avaliação do Modelo no Conjunto de Teste ---
print("\n--- Avaliação Final do Modelo SVM no Conjunto de Teste ---")

y_pred = best_svm_model.predict(X_teste)

# 5.1. Acurácia
accuracy = accuracy_score(y_teste, y_pred)
print(f"Acurácia no conjunto de teste: {accuracy:.4f}")

# 5.2. Relatório de Classificação (Precisão, Recall, F1-Score)
# As labels (0, 1, 2...) correspondem à ordem em que as classes foram mapeadas.
# Se você tiver uma lista de nomes de classes (ex: ['A', 'B', ..., 'nothing', 'space', 'del']),
# pode passá-las para target_names para um relatório mais legível.
print("\nRelatório de Classificação:")
print(classification_report(y_teste, y_pred))

# 5.3. Matriz de Confusão
print("\nMatriz de Confusão:")
# Para ConfusionMatrixDisplay, se o número de classes for grande, a visualização pode ficar apertada.
# Você pode ajustar o figsize ou salvar a imagem.

# Se você sabe os nomes das suas classes, pode passá-los para display_labels:
# Por exemplo: classes_names = [chr(ord('A') + i) for i in range(26)] + ['nothing', 'space', 'del']
# display_labels = classes_names

# Criar a matriz de confusão. Ajuste o figsize se tiver muitas classes.
disp = ConfusionMatrixDisplay.from_estimator(
    best_svm_model,
    X_teste,
    y_teste,
    cmap=plt.cm.Blues,
    # display_labels=display_labels, # Descomente e defina se tiver nomes de classes
    xticks_rotation='vertical' # Útil se os nomes das classes forem longos
)
disp.ax_.set_title("Matriz de Confusão do SVM")